In [ ]:
# Install required libraries
!pip install flask flask-ngrok flask-cors transformers google-play-scraper pyngrok

from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS
from pyngrok import ngrok
import pandas as pd
from google_play_scraper import Sort, reviews_all
from transformers import pipeline
import re

# Set ngrok authtoken (replace with your actual token)
ngrok.set_auth_token("2qFPxW9lzA3sZMjRILAkRAPftPv_4J3JCvRgQRtrkgcsYw8DP Paste ngrok auth token here")

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Start ngrok tunnel to the Flask app
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Function to extract app ID from the URL
def extract_app_id(url):
    match = re.search(r"id=([a-zA-Z0-9._-]+)", url)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid Google Play Store URL. Please ensure the URL contains an 'id' parameter.")

# Sentiment analysis pipeline
sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

# Function to generate reviews based on the URL (Google Play Store URL)
@app.route('/generate_reviews', methods=['POST'])
def generate_reviews():
    url = request.json.get("url")  # Get the URL from the frontend
    if not url:
        return jsonify({"error": "Please provide a valid Google Play Store URL."}), 400

    try:
        app_id = extract_app_id(url)
        # Fetch reviews for the extracted app ID
        hkr = reviews_all(app_id, sleep_milliseconds=0, lang='en', country='US', sort=Sort.NEWEST)

        # Create a DataFrame from the reviews
        df = pd.json_normalize(hkr)

        if df.empty:
            return jsonify({"error": "No reviews found for this app."}), 404

        # Preprocess the data
        columns_to_drop = [
            'reviewId', 'userImage', 'thumbsUpCount',
            'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion'
        ]
        df = df.drop(columns=columns_to_drop)
        df['content'] = df['content'].astype('str')

        # Now, return only the userName and content
        reviews_data = df[['userName', 'content']].to_dict(orient='records')

        return jsonify({"reviews": reviews_data})

    except ValueError as e:
        return jsonify({"error": str(e)}), 400


# Function to generate sentiment analysis results
@app.route('/generate_sentiment', methods=['POST'])
def generate_sentiment():
    url = request.json.get("url")  # Get the URL from the frontend
    if not url:
        return jsonify({"error": "Please provide a valid Google Play Store URL."}), 400

    try:
        app_id = extract_app_id(url)
        # Fetch reviews for the extracted app ID
        hkr = reviews_all(app_id, sleep_milliseconds=0, lang='en', country='US', sort=Sort.NEWEST)

        # Create a DataFrame from the reviews
        df = pd.json_normalize(hkr)

        if df.empty:
            return jsonify({"error": "No reviews found for this app."}), 404

        # Preprocess the data
        columns_to_drop = [
            'reviewId', 'userImage', 'thumbsUpCount',
            'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion'
        ]
        df = df.drop(columns=columns_to_drop)
        df['content'] = df['content'].astype('str')

        # Apply sentiment analysis
        df['result'] = df['content'].apply(lambda x: sentiment_analysis(x))
        df['sentiment'] = df['result'].apply(lambda x: x[0]['label'])
        df['score'] = df['result'].apply(lambda x: x[0]['score'])

        # Calculate sentiment percentages
        positive_count = df['sentiment'].value_counts().get('POSITIVE', 0)
        negative_count = df['sentiment'].value_counts().get('NEGATIVE', 0)

        positive = (positive_count / len(df)) * 100
        negative = (negative_count / len(df)) * 100

        # Decision logic
        if positive >= 70:
            decision = "The Application is worth downloading"
        elif 50 <= positive < 70:
            decision = "The Application is well received but consider your preferences"
        elif 40 <= positive < 50:
            decision = "The Application has a balanced reception. Consider your preferences and needs."
        else:
            decision = "There are significant issues with the Application."

        return jsonify({
            "positive_percentage": positive,
            "negative_percentage": negative,
            "decision": decision
        })

    except ValueError as e:
        return jsonify({"error": str(e)}), 400


# Run the app
if __name__ == '__main__':
    app.run(port=5000)
